In [49]:
"""Evaluation metric for Santa 2023."""

import pandas as pd
from ast import literal_eval
from dataclasses import dataclass
from sympy.combinatorics import Permutation
from typing import Dict, List


class ParticipantVisibleError(Exception):
    pass


def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        series_id_column_name: str,
        moves_column_name: str,
        puzzle_info_path: str)-> float:
     
    """Santa 2023 evaluation metric.

    Parameters
    ----------
    solution : pd.DataFrame

    submission : pd.DataFrame

    series_id_column_name : str

    moves_column_name : str

    Returns
    -------
    total_num_moves : int
    """
    if list(submission.columns) != [series_id_column_name, moves_column_name]:
        raise ParticipantVisibleError(
            f"Submission must have columns {series_id_column_name} and {moves_column_name}."
        )

    puzzle_info = pd.read_csv(puzzle_info_path, index_col='puzzle_type')
    total_num_moves = 0
    for sol, sub in zip(solution.itertuples(), submission.itertuples()):
        puzzle_id = getattr(sol, series_id_column_name)
        assert puzzle_id == getattr(sub, series_id_column_name)
        allowed_moves = literal_eval(puzzle_info.loc[sol.puzzle_type, 'allowed_moves'])
        allowed_moves = {k: Permutation(v) for k, v in allowed_moves.items()}
        puzzle = Puzzle(
            puzzle_id=puzzle_id,
            allowed_moves=allowed_moves,
            solution_state=sol.solution_state.split(';'),
            initial_state=sol.initial_state.split(';'),
            num_wildcards=sol.num_wildcards,
        )

        # Score submission row
        total_num_moves += score_puzzle(puzzle_id, puzzle, getattr(sub, moves_column_name))

    return total_num_moves


@dataclass
class Puzzle:
    """A permutation puzzle."""

    puzzle_id: str
    allowed_moves: Dict[str, List[int]]
    solution_state: List[str]
    initial_state: List[str]
    num_wildcards: int


def score_puzzle(puzzle_id, puzzle, sub_solution):
    """Score the solution to a permutation puzzle."""
    # Apply submitted sequence of moves to the initial state, from left to right
    moves = sub_solution.split('.')
    state = puzzle.initial_state
    for m in moves:
        power = 1
        if m[0] == "-":
            m = m[1:]
            power = -1
        try:
            p = puzzle.allowed_moves[m]
        except KeyError:
            raise ParticipantVisibleError(f"{m} is not an allowed move for {puzzle_id}.")
        state = (p ** power)(state)

    # Check that submitted moves solve puzzle
    num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
    if num_wrong_facelets > puzzle.num_wildcards:
        raise ParticipantVisibleError(f"Submitted moves do not solve {puzzle_id}.")

    # The score for this instance is the total number of moves needed to solve the puzzle
    return len(moves)


In [51]:

import pandas as pd
sample_submission = pd.read_csv("./data/sample_submission.csv")
solution = pd.read_csv("./data/puzzles.csv")
# score(solution,sample_submission,"id","moves","./data/puzzle_info.csv")


In [52]:
solution.shape

(398, 5)

In [60]:
puzzle_info = pd.read_csv("./data/puzzle_info.csv", index_col='puzzle_type')
moves = {}
for idx, row in puzzle_info.iterrows():
    allowed_moves = literal_eval(puzzle_info.loc[idx, 'allowed_moves'])
    allowed_moves = {k: Permutation(v) for k, v in allowed_moves.items()}
    moves[idx] = allowed_moves

In [61]:
moves['cube']

{'cube_2/2/2': {'f0': Permutation(23)(2, 19, 21, 8)(3, 17, 20, 10)(4, 6, 7, 5),
  'f1': Permutation(0, 18, 23, 9)(1, 16, 22, 11)(12, 13, 15, 14),
  'r0': Permutation(1, 5, 21, 14)(3, 7, 23, 12)(8, 10, 11, 9),
  'r1': Permutation(23)(0, 4, 20, 15)(2, 6, 22, 13)(16, 17, 19, 18),
  'd0': Permutation(6, 18, 14, 10)(7, 19, 15, 11)(20, 22, 23, 21),
  'd1': Permutation(23)(0, 1, 3, 2)(4, 16, 12, 8)(5, 17, 13, 9)},
 'cube_3/3/3': {'f0': Permutation(53)(6, 44, 47, 18)(7, 41, 46, 21)(8, 38, 45, 24)(9, 15, 17, 11)(10, 12, 16, 14),
  'f1': Permutation(53)(3, 43, 50, 19)(4, 40, 49, 22)(5, 37, 48, 25),
  'f2': Permutation(0, 42, 53, 20)(1, 39, 52, 23)(2, 36, 51, 26)(27, 29, 35, 33)(28, 32, 34, 30),
  'r0': Permutation(2, 11, 47, 33)(5, 14, 50, 30)(8, 17, 53, 27)(18, 24, 26, 20)(19, 21, 25, 23),
  'r1': Permutation(53)(1, 10, 46, 34)(4, 13, 49, 31)(7, 16, 52, 28),
  'r2': Permutation(53)(0, 9, 45, 35)(3, 12, 48, 32)(6, 15, 51, 29)(36, 38, 44, 42)(37, 41, 43, 39),
  'd0': Permutation(15, 42, 33, 24)(1

In [54]:
puzzle_info.shape

(26, 1)

In [59]:
pd.DataFrame(solution.groupby(by='puzzle_type').count())

,id,solution_state,initial_state,num_wildcards
puzzle_type,,,,
cube_10/10/10,5,5,5,5
cube_19/19/19,4,4,4,4
cube_2/2/2,30,30,30,30
cube_3/3/3,120,120,120,120
cube_33/33/33,3,3,3,3
cube_4/4/4,60,60,60,60
cube_5/5/5,35,35,35,35
cube_6/6/6,12,12,12,12
cube_7/7/7,5,5,5,5


In [20]:
(Permutation([1,0,2,3]) ** (-1))(['A','B','C','D'])

['B', 'A', 'C', 'D']

In [22]:
(Permutation([1,0,2,3]) ** (1))(['A','B','C','D'])

['B', 'A', 'C', 'D']

In [36]:
m = (Permutation([0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11, 12, 13, 14, 15, 16, 20, 18, 21, 10, 8, 22, 23]) ** 1) 
m

Permutation(23)(2, 19, 21, 8)(3, 17, 20, 10)(4, 6, 7, 5)

In [38]:
(m**-1)(['A','A','A','A','B','B','B','B','C','C','C','C','D','D','D','D','E','E','E','E','F','F','F','F'])


['A',
 'A',
 'C',
 'C',
 'B',
 'B',
 'B',
 'B',
 'F',
 'C',
 'F',
 'C',
 'D',
 'D',
 'D',
 'D',
 'E',
 'A',
 'E',
 'A',
 'E',
 'E',
 'F',
 'F']

In [33]:
p = Permutation([0, 2, 1])
q = Permutation([2, 1, 0])

In [35]:
(p-1)(['a','b','c'])

['a', 'c', 'b']

In [9]:
(q)((p)(['a','b','c']))

['b', 'c', 'a']

In [21]:
[i^q for i in [0,2,1]]

[2, 0, 1]

In [13]:
(q)(p)(['a,'b','c'])

ValueError: a is not an integer